In [33]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from time import sleep

In [34]:
from infra.Controllers.ProcessoController import ProcessoController
from infra.Controllers.PessoaController import PessoaController
from infra.Controllers.ParteController import ParteController
from infra.Controllers.AdvogadoController import AdvogadoController
from infra.Controllers.AudienciaController import AudienciaController

In [35]:
processo_controller = ProcessoController()
pessoa_controller = PessoaController()
parte_controller = ParteController()
advogado_controller = AdvogadoController()
audiencia_controller = AudienciaController()

In [36]:
class Dados:
    def __init__(self):
        self.drive = webdriver.Chrome()
    
    def login(self,trt):
        self.trt = trt
        self.drive.get(f"https://pje.trt{self.trt}.jus.br/primeirograu/authenticateSSO.seam"); input()

    def busca_tabela(self, datafinal, datainicio=None):
        self.drive.get(f"https://pje.trt{self.trt}.jus.br/primeirograu/ProcessoAudiencia/PautaAudiencia/listView.seam")

        datas = self.drive.find_elements(By.XPATH,"//input[@class='rich-calendar-input dtIniciomaskDate']")
        self.drive.execute_script(f"arguments[0].setAttribute('value', '{datafinal}')", datas[1])
        self.drive.execute_script(f"arguments[0].setAttribute('value', '{datainicio}')", datas[0])

        self.drive.find_element(By.ID,"processoAudienciaSearchForm:searchButton").click();sleep(1.5)
        self.drive.find_element(By.CLASS_NAME,"rich-table").find_element(By.CLASS_NAME,"rich-table-thead").find_elements(By.TAG_NAME,"th")[1].find_element(By.TAG_NAME,"a").click();sleep(1)

        rows = int(self.drive.find_element(By.XPATH, '//*[@id="idProcessoAudienciaPanel_body"]/span').text.split()[2])
        pages = rows//15 + (rows%15!=0)

        print(f"No trt{self.trt} tem {rows} audiencias")

        for page in range(pages):
            table = self.drive.find_element(By.XPATH,'//*[@id="idProcessoAudiencia"]')

            for row in table.find_elements(By.TAG_NAME,"tr")[1::2]:
                data = [column.text for column in row.find_elements(By.TAG_NAME,"td")][3:8]

                dia, hora = data[0].split()
                dia = "-".join(dia.split("-")[-1])
                numero = data[1]
                tipo = data[4]

                processo = processo_controller.select(numero=numero)
                if processo: processo = processo[0]
                else: processo = self.cadastroProcesso(numero)

                audiencia = audiencia_controller.select(processo_id=processo.id, dia=dia, hora=hora)
                if audiencia: continue

                audiencia = audiencia_controller.insert(
                    dia=dia,
                    hora=hora,
                    tipo=tipo,
                    processo_id=processo.id
                )

                audiencia.addParte()

            if page+1 != pages: self.drive.find_elements(By.CLASS_NAME,"rich-inslider-td-arrow")[1].click();sleep(1)
    

    def cadastroProcesso(self, numero):
        #self.drive.get("https://pje.trt16.jus.br/primeirograu/Painel/painel_usuario/advogado.seam")
        #driver.execute_script("window.open('https://www.outroexemplo.com', '_blank');")

        self.drive.execute_script(f"window.open('https://pje.trt{self.trt}.jus.br/primeirograu/Painel/painel_usuario/advogado.seam', '_blank');")

        self.drive.find_element(By.ID, "leftAdvPnl_header_label").click()
        element = self.drive.find_element(By.ID, "consultaProcessoAdvogadoForm:numeroProcessoDecoration:numeroProcesso")
        sleep(0.5)

        element.get_attribute('value')
        self.drive.execute_script(f"arguments[0].value = '{numero}';", element)

        self.drive.execute_script("arguments[0].dispatchEvent(new Event('change'));", element)
        sleep(1)
        self.drive.find_element(By.ID, "consultaProcessoAdvogadoForm:searchButon").click()
        sleep(1)

        link = self.drive.find_element(By.XPATH, f"//a[span[contains(text(), '{numero}')]]").get_attribute('onclick')
        link = link[link.find("https"):link.find("')")]
        self.drive.get(link); sleep(1)

        self.drive.find_element(By.XPATH, "//span[@class = 'texto-numero-processo']//button").click()
        sleep(1)

        vara = self.drive.find_element(By.XPATH, "//div[@aria-label = 'Dados do processo']//dd[1]").text
        classe, numero = self.drive.find_element(By.XPATH, "//div[@aria-label = 'Dados do processo']//dd[2]").text.split()
        valor = self.drive.find_element(By.XPATH, "//div[@aria-label = 'Dados do processo']//dd[5]").text.split()[1]

        processo = processo_controller.select(numero=numero)
        if processo: processo = processo[0]
        else:
            processo = processo_controller.insert(
                numero=numero,
                valor=valor.replace('.', '').replace(',', '.'),
                vara=vara,
                classe=classe,
                link=link
            )

        partes = self.drive.find_elements(By.XPATH, "//div[@class= 'is-item-pilha-parte']")
        for parte in partes:
            polo = parte.find_element(By.CLASS_NAME, "polo-header").text.split()[1]
            print(polo)

            pessoas = parte.find_elements(By.TAG_NAME, "ul")[::2]
            print(len(pessoas))
            for pessoa in pessoas:
                nome = pessoa.find_element(By.TAG_NAME, "pje-nome-parte").text
                pessoa_dados = [x.text for x in pessoa.find_elements(By.CLASS_NAME, "ng-star-inserted")[:2]]
                cpf_cnpj = pessoa_dados[0].split()[1] if pessoa_dados[0] != "" else pessoa_dados[1].split()[1]

                Pessoa = pessoa_controller.select(nome=nome,cpf_cnpj=cpf_cnpj)
                if Pessoa: Pessoa = Pessoa[0]            
                else: Pessoa = pessoa_controller.insert(nome=nome, cpf_cnpj=cpf_cnpj)
                print(Pessoa)

                tipo = pessoa.find_element(By.TAG_NAME, "li").text.split()[0][:-1]

                parte = parte_controller.select(pessoa_id=Pessoa.id, processo_id=processo.id)
                if parte: parte = parte[0]
                else:
                    parte = parte_controller.insert(
                        tipo=tipo,
                        polo=polo,
                        processo_id=processo.id,
                        pessoa_id=Pessoa.id
                    )


                for adv in pessoa.find_elements(By.CLASS_NAME, "partes-representante"):
                    advogado = adv.text
                    if "ADVOGADO" not in advogado: continue

                    nome, cpf, aob = advogado.split("\n")[:3]
                    nome = nome[:nome.find(" (ADVOGADO)")]
                    aob = aob.split()[1][:-1]

                    advogado = advogado_controller.select(aob=aob)
                    if advogado: advogado = advogado[0]
                    else: advogado = advogado_controller.insert(nome=nome, aob=aob)

                    print(advogado)
                    parte.addAdvogado(advogado)
        self.drive.close()
        return processo

In [37]:
dados = Dados()
trts = [2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 15, 16, 18, 19, 22, 23]

In [ ]:
while trts:
    try:
        trt = trts.pop()
        dados.login(trt)
        dados.busca_tabela('11/10/2024',"08/10/2024")
    except NoSuchElementException:
        print("No trt" + str(trt) + " ocorreu um erro")
        trts.append(trt)
    